## Summer Olympics Ranking Medals
Author: Damian Zamora Rivas
<br>This script is part of my article wrote in my blog in Medium: **https://damzar.medium.com/**

If you want to read how was used and an analysis of the results of this script, the article links are:
* **English version**: https://link.medium.com/LooeltlSxib
* **Spanish version**: https://link.medium.com/9vlsUYoSxib

## Part One

### Data preparation

This part consists of the following steps:
* Downloading of the ranking of medals and adding to a dataframe with all countries
* Preparation of the shape file of all countries to be plotted as a map, with medal score

In [ ]:
import pandas as pd

In [ ]:
#Function to read and save the tables from a wikipedia page.
#Input variable: Language ('en' for english, 'es' for spanish), exact term (string) to search (name of the article)
#  and a dictionary with UTF-8 code.
#Output variable: List of tables from the web page.
#NOTE: It is important to have the dictionary wikicode in the same script.

def wikipedia_tables(language, article, wikicode):
    
    if language == 'en':
        wiki = 'https://en.wikipedia.org'
    if language == 'es':
        wiki = 'https://es.wikipedia.org'
    
    termlist = list()
    for letter in article:
        termlist.append(letter)
        for k in range(len(wikicode)):
            if letter == wikicode.iloc[k,0]:
                termlist.remove(letter)
                termlist.append(wikicode.iloc[k,1])
            else:
                continue
                
    term = "".join(termlist)
    url = wiki + '/wiki/' + term
    wikipage = pd.read_html(url)
    
    return wikipage

In [ ]:
#First at all, from a page I extract the url-8 code to later do the changes of characters, for instance 'ñ'.
pagutf8 = pd.read_html('https://www.w3schools.com/tags/ref_urlencode.asp')
utf8 = pagutf8[0]
utf8.rename(columns = {'From Windows-1252':'Windows-1252', 'From UTF-8':'UTF-8'}, inplace = True)

wikicode = pd.DataFrame()
utfdict = dict()

#In this section from all url-8 code I select the characters needed, and save them in a dictionary.
i = 0

for row in utf8.iloc[:,0]:
    i = i +1
    if row == 'á' or row == 'é' or row == 'í' or row == 'ó' or row == 'ú':
        utfdict[row] = utf8.iloc[i-1,2]
    elif row == 'Á' or row == 'É' or row == 'Í' or row =='Ó' or row == 'Ú':
        utfdict[row] = utf8.iloc[i-1,2]
    elif row == 'ñ' or row == 'Ñ':
        utfdict[row] = utf8.iloc[i-1,2]
    elif row == '¡' or row == '!' or row == '¿' or row == '?' or row == ':':
        utfdict[row] = utf8.iloc[i-1,2]
    else:
        continue

#Dictionary to write the correct form of the link to read.
utfdict[' '] = '_'
wikicode = pd.DataFrame([[key, utfdict[key]] for key in utfdict.keys()], columns=['Character', 'Code'])

#English page of Olympic medal table.
article = '2020 Summer Olympics medal table'
wikipage = wikipedia_tables('en', article, wikicode)

In [ ]:
#Separation of country code from country name (Country (CTR)) to put them in different columns.

#These lists will be used to do the dataframe of medals.
countries = []
countries_co = []

i = 0
for cou in wikipage[1]['Team']:    #cou means country
    i += 1
    wl = []                          #wl means word list
    cl = []                          #cl means code list
    
    j = 0
    for l in cou:                    #l means letter
        j += 1
        if l != '(':
            wl.append(l)
        elif l == '(':
            cl.append(cou[j])
            cl.append(cou[j+1])
            cl.append(cou[j+2])
            wl.pop()
            break
            
    country = "".join(wl)
    code = "".join(cl)
    
    countries.append(country)
    countries_co.append(code)

#There is an extra item in each list, so it will be deleted.
countries.pop()
countries_co.pop()

#Dataframe of medals
medals_df = wikipage[1]
medals_df.drop(['Rank','Team'], axis = 1, inplace = True)
medals_df.drop([len(medals_df)-1],axis = 0, inplace = True)

medals_df.insert(0,'country',countries)
medals_df.insert(1,'code',countries_co)
medals_df.rename(columns = {'Gold':'gold','Silver':'silver','Bronze':'bronze','Total':'total'},
                inplace = True)

To use this section it is necessary to download the shape file of the countries, which is in my Kaggle account: https://www.kaggle.com/damizar/a-fairer-ranking-medalist-countries?select=countries_shape 
<br>*It was originally downloaded from Natural Earth Data web page*. The useful library to work with shape files is **geopandas**.

The shape file that is used has many columns, of which only three will be useful to this work. Remember to read only the file with .shp extension, which have to be in a folder with more files.

Some countries names are different to the IOC names, for example 'Chinese Taipei' and 'Taiwan', so a little loop is used to know which ones are, and use them to add to the dataframe which will be plotted as a map. This loop is used a pair of times in the script. I decide to write it in this Markdown:
>world_win = list(world_df['country']) <br> for lost in medals_df['country']:<br> if (lost in world_win) == True: <br>     continue <br>else: <br> print(lost)

Later, to get the amount of athletic people per country, it is necessary to get the information from the corresponding wikipedia page.

In [ ]:
import geopandas as gpd

In [ ]:
shapefile = 'countries_shape/ne_10m_admin_0_countries_lakes.shp'
countries_df = gpd.read_file(shapefile)[['NAME', 'geometry']]  #Reading only the columns needed.

countries_df.columns = ['country', 'geometry']  #Renaming columns with the same nomenclature.
world_df = pd.DataFrame(countries_df['country']) #saving dataframe without geometry

In [ ]:
#Extraction of athletic people per country.
article = '2020 Summer Olympics'
page = wikipedia_tables('en', article, wikicode)
raw_athletes = page[7]

#After see the dataframe, removing of the first two and last rows, knowing column with indexes 1 and 2 are the important.
raw_athletes = raw_athletes.drop(world_df.index[0])
raw_athletes = raw_athletes.drop(world_df.index[1])
raw_athletes = raw_athletes.drop(world_df.index[len(raw_athletes)])

#Adding new column to the dataframes.
medals_df.insert(1,'athletes',0)

In [ ]:
#Including of the number athletes to the dataframe with only the countries which have won a medal
for co in medals_df['country']:  
    
    if co == 'Japan':
        index_w = medals_df[medals_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Japan (Host)'].index.item()
        medals_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
    
    for co_aux in raw_athletes[1]:
        if co == co_aux:
            message = 'Match'
            break
        else:
            message = "Don't match"
            continue
    
    if message == "Don't match":
        continue
    else:
        index_w = medals_df[medals_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]==co].index.item()
        medals_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue

In [ ]:
#For some reason, the column Athletes is type object, which makes it impossible to use its integer data, so
# is changed to type int.
df_aux = medals_df[['athletes']].astype(int)
list_a = df_aux['athletes']
medals_df.insert(7,'athl',list_a)
medals_df.drop(['athletes'], axis=1, inplace = True)
medals_df.rename(columns={'athl': 'athletes'}, inplace=True)

# Part Two

### Calculation

This part consists of the following steps:

* Modification of the value of each medal, according to a weight, to obtain a medal score
* Calculation of the ratio Medal score / Athletes per country 
* Web scraping of the GDP per capita of countries with medals
* Calculation of the GDP average and normalization of data
* Rescaling of the differences between GDP average and each GDP per capita
* Sum of both ratio m/a and rescaled values, and multiplication by 100 to get the final score


It is important to know the string to search on the wikipedia page, which is GDP Per Capita.
<br>It is necessary an exploratory searching in pages to know how is the string, which is '•\xa0Per capita', where \xa0 is an space.

In [ ]:
#Modification of the value of each medal, according to a weight.
medals_df['gold_w'] = medals_df['gold']*3
medals_df['silver_w'] = medals_df['silver']*2
medals_df['bronze_w'] = medals_df['bronze']
medals_df['total_w'] = medals_df['gold_w'] + medals_df['silver_w'] + medals_df['bronze_w']

#Getting of the ratio = madals/number of athletes per country
#'m' means medals and 'a' means number of athletes per country
medals_df['ratio_m/a'] = medals_df['total_w'].div(medals_df['athletes'])

In [ ]:
percapita = '•\xa0Per capita'
list_aux = list(medals_df['country'])
raw_percap = []
countries = []

for country in list_aux:

    if country == 'ROC':
        article = 'Russia'
    elif country == 'Chinese Taipei':
        article = 'Taiwan'
    else:
        article = country

    page = wikipedia_tables('en', article, wikicode)
    df_aux = page[0]
    
    notice = 'nothing'
    i = 0
    
    for item in df_aux.iloc[:,0]:
        i = i + 1
        if item == percapita and notice == 'nothing':
            notice = 'first GDP'
        if item == percapita and notice == 'first GDP':
            per_capita = df_aux.iloc[i-1,1]
    
    countries.append(country)
    raw_percap.append(per_capita)

df_percapita = pd.DataFrame({'country':countries, 'raw_percapita':raw_percap})

In [ ]:
#Cleaning of the raw_percapita column
percapita = []

for gdp in df_percapita['raw_percapita']:
    if '[' in gdp:
        money = gdp.split('[')[0]
    else:
        money = gdp.split('(')[0]
    
    if 'U' in money:
        money = money.replace('US','')
    
    if '(' in money:
        money = money.split('(')[0]
        
    aux = money.replace('$','')
    number = int(aux.replace(',',''))
    
    percapita.append(number)
    
df_percapita['per_capita'] = percapita

#Getting of mean an maximum values
mean_perc = int(df_percapita['per_capita'].sum()/len(df_percapita))
max_perc = max(df_percapita['per_capita'])
min_perc = min(df_percapita['per_capita'])

In [ ]:
fac = 1/max_perc
df_percapita['n_percap'] = df_percapita['per_capita']*fac
mean_nperc = df_percapita['n_percap'].sum()/len(df_percapita)

df_percapita['difference'] = mean_nperc - df_percapita['n_percap']

max_nperc = max(df_percapita['difference'])
min_nperc = min(df_percapita['difference'])

#Scale based in the different GDP per capita per country
df_percapita['scale'] = 0 + (df_percapita['difference'] - min_nperc)

#It is time to add some columns from df_percapita (which was a helping dataframe) to the main dataframe
medals_df['per_capita'] = df_percapita['per_capita']
medals_df['n_percap'] = df_percapita['n_percap']
medals_df['difference'] = df_percapita['difference']
medals_df['scale'] = df_percapita['scale']

#Sum of the ratio m/a and the scale based in GDP per capita. 
medals_df['score_dec'] = medals_df['ratio_m/a'] + medals_df['scale']

#Just for get clearer differences between scores, previous score is multiplied for 100
medals_df['score'] = medals_df['score_dec'] * 100

In [ ]:
#Saving of the ranking with all process (only countries with medals).
medals_df.to_csv('medals_ranking.csv', index = False)

# Part Three

### Data Visualization:

* Chart of GDP per capita of countries with medals
* Original map of medals
* Final map of medals

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as tick

In [ ]:
#Chart of GDP per capita of countries with medals ordering according to the ranking

def y_fmt(x, y):
    return '${:,.0f}'.format(x)

title = 'GDP per capita of the medalist countries'

fig = plt.figure(figsize=(23, 8))
ax = fig.add_subplot(111)
plt.grid(axis = 'x')
ax.set(ylim=(0, 90000))

plt.title(title + '\n', fontsize=21)

plt.xlabel('Countries', fontsize=17)
plt.ylabel('PIB per capita (USD)', fontsize=17)
plt.xticks(rotation=45, horizontalalignment='right')

plt.text(60, 21100, 'Average = ${:,.0f}'.format(mean_perc), fontsize=13)
ax.hlines(mean_perc, -5, 90, linestyles='dashed', colors='red')

parameters = {'ytick.labelsize': 13, 'xtick.labelsize': 9}
plt.rcParams.update(parameters)

plt.plot(medals_df['country'], medals_df['per_capita'], 'bs-')

ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))

plt.savefig('File_name.ext', dpi = 200, bbox_inches = 'tight')

In [ ]:
medals_df.sort_values('per_capita', ascending=False, inplace=True)

In [ ]:
#Chart of GDP per capita of countries with medals ordering from highest to lowest GDP

def y_fmt(x, y):
    return '${:,.0f}'.format(x)

title = 'GDP per capita of the medalist countries'

fig = plt.figure(figsize=(23, 8))
ax = fig.add_subplot(111)
plt.grid(axis = 'x')
ax.set(ylim=(0, 90000))

plt.title(title + '\n', fontsize=21)

plt.xlabel('Countries', fontsize=17)
plt.ylabel('PIB per capita (USD)', fontsize=17)
plt.xticks(rotation=45, horizontalalignment='right')

ax.annotate('${:,.0f}'.format(max_perc), fontsize= 14, xy=(0.3, 86500), xytext=(4, 85900), 
            arrowprops=dict(facecolor='b', shrink=0.05))
ax.annotate('${:,.0f}'.format(min_perc), fontsize= 14, xy=(92, 1500), xytext=(90.7, 10500), 
            arrowprops=dict(facecolor='b', shrink=0.01))

plt.text(57, 21100, 'Average = ${:,.0f}'.format(mean_perc), fontsize=15)
ax.hlines(mean_perc, -5, 95, linestyles='dashed', colors='red')

parameters = {'ytick.labelsize': 15, 'xtick.labelsize': 9}
plt.rcParams.update(parameters)

plt.plot(medals_df['country'], medals_df['per_capita'], 'bs-')

ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))

plt.savefig('File_name.ext', dpi = 200, bbox_inches = 'tight')

In [ ]:
medals_df.sort_values('score', ascending=False, inplace=True)

In [ ]:
#Chart of countries with medals, ordered to the new score

title = 'Score of medalist countries'

fig = plt.figure(figsize=(23, 8))
ax = fig.add_subplot(111)
plt.grid()
ax.set(ylim=(0, 250))

plt.title(title + '\n', fontsize=21)

plt.xlabel('Countries', fontsize=17)
plt.ylabel('Score', fontsize=17)
plt.xticks(rotation=45, horizontalalignment='right')

max_score = int(max(medals_df['score']))
min_score = int(min(medals_df['score']))
ax.annotate(max_score, fontsize= 14, xy=(0.3, 243), xytext=(4, 241), 
            arrowprops=dict(facecolor='b', shrink=0.05))
ax.annotate(min_score, fontsize= 14, xy=(91.5, 21), xytext=(87, 18.5), 
            arrowprops=dict(facecolor='b', shrink=0.01))

ax.hlines(mean_perc, -5, 95, linestyles='dashed', colors='red')

parameters = {'ytick.labelsize': 15, 'xtick.labelsize': 9}
plt.rcParams.update(parameters)

plt.plot(medals_df['country'], medals_df['score'], 'bs-')

plt.savefig('File_name.ext', dpi = 200, bbox_inches = 'tight')

Creating of the maps with geodataframe world_df and geodataframe countries_df 
<br> First it has to complete the world_df with data from dataframe medals_df
<br> And then this dataframe merge with the geodataframe

In [ ]:
#'total' and 'total_w' refer to total of won medals, and total of medal score after give a weight to each medal
world_df[['total','total_w','ratio_m/a','score']] = 0 

for co in world_df['country']:  #co means country
    
    if co == 'Taiwan':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='Chinese Taipei'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
    
    if co == 'Russia':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='ROC'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
        
    if co == 'Czechia':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='Czech Republic'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
    
    if co == 'United Kingdom':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='Great Britain'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
        
    if co == 'United States of America':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='United States'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
    
    if co == 'Dominican Rep.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='Dominican Republic'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
    
    if co == "Côte d'Ivoire":
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='Ivory Coast'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
        
    if co == "Macedonia":
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']=='North Macedonia'].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue
    
    for co_aux in medals_df['country']:
        if co == co_aux:
            message = 'Match'
            break
        else:
            message = "Don't match"
            continue
    
    if message == "Don't match":
        continue
    else:
        index_w = world_df[world_df['country']==co].index.item() 
        index_m = medals_df[medals_df['country']==co].index.item()
        world_df.loc[index_w,'total'] = medals_df.loc[index_m,'total']
        world_df.loc[index_w,'total_w'] = medals_df.loc[index_m,'total_w']
        world_df.loc[index_w,'ratio_m/a'] = medals_df.loc[index_m,'ratio_m/a']
        world_df.loc[index_w,'score'] = medals_df.loc[index_m,'score']
        continue

#Removing row corresponding to 'Antarctica'.
ind = world_df[world_df['country']=='Antarctica'].index.item()
world_df = world_df.drop(world_df.index[ind])

In [ ]:
#Just for fun, I add the number of athletes, and to see all data in the same dataframe
world_df.insert(1,'athletes',0)
#Refugee Olympic Team is not included

for co in world_df['country']:  #co means country
    
    if co == 'United States of America':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='United States'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Japan':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Japan (Host)'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'United Kingdom':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Great Britain'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Russia':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='ROC'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Czechia':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Czech Republic'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
    
    if co == 'Taiwan':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Chinese Taipei'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
    
    if co == 'Dominican Rep.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Dominican Republic'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
    
    if co == "Côte d'Ivoire":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Ivory Coast'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Macedonia":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='North Macedonia'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Bosnia and Herz.":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Bosnia and Herzegovina'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Dem. Rep. Congo":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Democratic Republic of the Congo'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Antigua and Barb.":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Antigua and Barbuda'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Cabo Verde":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Cape Verde'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Cook Is.":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Cook Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Cayman Is.":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Cayman Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "eSwatini":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Eswatini'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "Gambia":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='The Gambia'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == "U.S. Virgin Is.":
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Virgin Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'British Virgin Is.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='British Virgin Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Timor-Leste':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='East Timor'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Eq. Guinea':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Equatorial Guinea'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Micronesia':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Federated States of Micronesia'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Congo':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Republic of the Congo'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'St. Vin. and Gren.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Saint Vincent and the Grenadines'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'São Tomé and Principe':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='São Tomé and Príncipe'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Solomon Is.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Solomon Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Central African Rep.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Central African Republic'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'Marshall Is.':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Marshall Islands'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'St. Kitts and Nevis':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='Saint Kitts and Nevis'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
        
    if co == 'S. Sudan':
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]=='South Sudan'].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue
    
    for co_aux in raw_athletes[1]:
        if co == co_aux:
            message = 'Match'
            break
        else:
            message = "Don't match"
            continue
    
    if message == "Don't match":
        continue
    else:
        index_w = world_df[world_df['country']==co].index.item() 
        index_a = raw_athletes[raw_athletes[1]==co].index.item()
        world_df.loc[index_w,'athletes'] = raw_athletes.loc[index_a,2]
        continue

In [ ]:
#Saving of the ranking including all countries
world_df.to_csv('world_olympics.csv', index = False)

In [ ]:
#Removing row corresponding to 'Antarctica' to the geodataframe.
ind = countries_df[countries_df['country']=='Antarctica'].index.item()
countries_df = countries_df.drop(countries_df.index[ind])

#Merge of dataframe and geodataframe
map_geodf = countries_df.merge(world_df, on='country')

In [ ]:
#Thematic map of countries according their medals
title = 'Map of countries with medals'

fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)

plt.title(title, fontsize=21)

map_geodf.plot(column='total',  cmap = 'Set1_r', ax = ax)

ax.axis('off')

min_med = min(map_geodf['total'])
max_med = max(map_geodf['total'])
bar = plt.cm.ScalarMappable(cmap='Set1_r', norm=plt.Normalize(vmin=min_med, vmax=max_med))

# empty array for the data  range
bar._A = []
cax = plt.axes([0.9, 0.3, 0.015, 0.4])  #[xcoord, ycoord, width, long]

# add the colorbar to the figure
cbar = fig.colorbar(bar, cax=cax)
cbar.set_label('Number of Medals', fontsize = 12)

ax.annotate('Country with more medals\nUnited States (108)',xy=(0.1, .18),  xycoords='figure fraction', 
            horizontalalignment='left', verticalalignment='top', fontsize=11)

plt.savefig('File_name.ext', dpi = 200, bbox_inches = 'tight')

In [ ]:
#Thematic map of countries according their scores
title = 'Map of countries with final score'

fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)

plt.title(title, fontsize=21)

map_geodf.plot(column='score',  cmap = 'Set1_r', ax = ax)

ax.axis('off')

min_med = min(map_geodf['score'])
max_med = max(map_geodf['score'])
bar = plt.cm.ScalarMappable(cmap='Set1_r', norm=plt.Normalize(vmin=min_med, vmax=max_med))

# empty array for the data  range
bar._A = []
cax = plt.axes([0.9, 0.3, 0.015, 0.4])  #[xcoord, ycoord, width, long]

# add the colorbar to the figure
cbar = fig.colorbar(bar, cax=cax)
cbar.set_label('Score', fontsize = 12)

ax.annotate('Country with biggest score\nBermuda (241)',xy=(0.1, .18),  xycoords='figure fraction', 
            horizontalalignment='left', verticalalignment='top', fontsize=11)

plt.savefig('File_name.ext', dpi = 200, bbox_inches = 'tight')

In [ ]:
map_geodf.to_csv('world_polygons.csv')